In [2]:
%load_ext autoreload
%autoreload 2

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [3]:
import os
import sys

root_dir = os.path.abspath(os.path.join(os.getcwd(), '../..'))
print(root_dir)
assert os.path.exists(root_dir), f'Could not find root directory at {root_dir}'
sys.path.insert(0, root_dir)

from custom_utils.config_handler import read_config, pprint_config

/Users/user010/Desktop/Programming/ML/En2RuTranslator


In [4]:
overrides = ["setup=prod"]
cfg = read_config(overrides=overrides)
pprint_config(cfg)

{
  "root": "/Users/user010/Desktop/Programming/ML/En2RuTranslator",
  "params": {
    "batch_size": 16,
    "max_length": 512,
    "train_args": {
      "evaluation_strategy": "epoch",
      "learning_rate": 2e-05,
      "per_device_train_batch_size": 16,
      "per_device_eval_batch_size": 16,
      "weight_decay": 0.01,
      "save_total_limit": 3,
      "num_train_epochs": 4,
      "predict_with_generate": true
    },
    "wandb_args": {
      "report_to": "wandb",
      "run_name": "finetune"
    }
  },
  "dest_synonym_searcher": {
    "path": "/Users/user010/Desktop/Programming/ML/En2RuTranslator/models/embs/cc.ru.100.bin",
    "type": "fasttext"
  },
  "src_synonym_searcher": {
    "path": "/Users/user010/Desktop/Programming/ML/En2RuTranslator/models/embs/cc.en.100.bin",
    "type": "fasttext"
  },
  "translator": {
    "name": "opus-distilled-en-ru",
    "model_and_tokenizer_name": "under-tree/transformer-en-ru",
    "output_dir": "/Users/user010/Desktop/Programming/ML/En2RuTra

In [6]:
%%time
from translator_app import TranslatorApp

app = TranslatorApp(cfg)

/Users/user010/Desktop/Programming/ML/En2RuTranslator/venv/lib/python3.11/site-packages/transformers/models/marian/tokenization_marian.py:197: UserWarning: Recommended: pip install sacremoses.
  warnings.warn("Recommended: pip install sacremoses.")


CPU times: user 4.22 s, sys: 2.59 s, total: 6.81 s
Wall time: 8.32 s


In [7]:
%%time
text = "Oh, dear, I'm afraid I've said too much. I'm a bad, bad blogger."
app_output = app(text)

/Users/user010/Desktop/Programming/ML/En2RuTranslator/venv/lib/python3.11/site-packages/transformers/tokenization_utils_base.py:3848: UserWarning: `as_target_tokenizer` is deprecated and will be removed in v5 of Transformers. You can tokenize your labels by using the argument `text_target` of the regular `__call__` method (either in the same call as your input texts if you use the same keyword arguments, or in a separate call.
  warnings.warn(


CPU times: user 9.32 s, sys: 1.47 s, total: 10.8 s
Wall time: 11.3 s


### App Output

* input words
* input tokens
* src synonyms
* output words
* output tokens
* dest synonyms
* translation 
* attention*

```python
{
            'translation': str,
            'input_tokens': list[str]
            'input_words': list[str]
            'output_tokens': list[str]
            'output_words': list[str]
            'src_synonyms': list[list[str]]
            'dest_synonyms': list[list[str]]
            'attention': list[list[float]]
}
```

In [81]:
print(app_output['input_words'])

['oh', ',', 'dear', ',', 'i', "'m", 'afraid', 'i', "'ve", 'said', 'too', 'much', '.', 'i', "'m", 'a', 'bad', ',', 'bad', 'blogger', '.']


In [8]:
print(app_output['input_tokens'])

['▁Oh', ',', '▁dear', ',', '▁I', "'", 'm', '▁afraid', '▁I', "'", 've', '▁said', '▁too', '▁much', '.', '▁I', "'", 'm', '▁a', '▁bad', ',', '▁bad', '▁blogger', '.']


In [9]:
dear_idx = app_output['input_words'].index('dear')
print(app_output['src_synonyms'][dear_idx])

['dearest', 'dears', 'dear.And', 'darling', 'dearies']


In [10]:
print(app_output['translation'])

О, дорогая, боюсь, я слишком много сказал. Я плохой, плохой блоггер.
